# Behavioral Cloning

In [7]:
import cv2 as cv
import os
import time
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Simple Network
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D


In [2]:
pathdata = "../data/"
# data_dir = './data/mydata'
# data_csv = '/driving_log.csv'
# model_json = 'model_behavioral_cloning_augmentation.json'
# model_weights = 'model_behavioral_cloning_augmentation.h5'

cnames = ['centerIm', 'leftIm','rightIm','steer','throttle','brake','speed']
df = pd.read_csv(pathdata+"driving_log.csv", names=cnames)
# df.head()

# Read center image
xCenter = df['centerIm']
xLeft  = df['leftIm']
xRight = df['rightIm']
ysteer = df['steer']
del df


In [3]:
# Multiple (camera) images as input
# Correct left and 

def read3camerasImg(pathCenter, pathLeft, pathRight, steering, steerAdj = 0.2):
    """
    input: 
        "pathCenter", "pathLeft" and "pathRight" are all (nx1) vector of strings, containing the path
        of the saved image
        example: pathCenter[0], pathLeft[0] and pathRight[0]
        C:\SelfDrivingCarNanodegree\3BehavioralCloning\data\IMG\center_2018_04_05_13_39_08_155.jpg
        C:\SelfDrivingCarNanodegree\3BehavioralCloning\data\IMG\left_2018_04_05_13_39_08_155.jpg
        C:\SelfDrivingCarNanodegree\3BehavioralCloning\data\IMG\right_2018_04_05_13_39_08_155.jpg
        
        "steering" is a vector of double for the steering angle. Left steering is positive, right is negative
        example: steeing[0] = -0.36
    
    """
    assert len(pathCenter) == len(pathLeft) == len(pathRight) == len(steering)
    images3 = []
    steer = []

    nim = len(xCenter)
    for i in range(nim):
        imgC = cv.cvtColor(cv.imread(xCenter[i]), cv.COLOR_BGR2RGB)
        imgL = cv.cvtColor(cv.imread(xLeft[i]), cv.COLOR_BGR2RGB)
        imgR = cv.cvtColor(cv.imread(xRight[i]), cv.COLOR_BGR2RGB)
        
        # convert 
        # correct steering angle (y) for lect image and right image
        yC = steering[i]
        yL = steering[i] + steerAdj
        yR = steering[i] - steerAdj
        images3.extend([imgC, imgL, imgR])
        steer.extend([yC, yL, yR])
        if (i%1000==0): print(i)
        
    y = np.array(steer)
    X = np.array(images3)
    
    return X, y



In [53]:
def flipBatchImg(images, measurement):
    """
    Create a new Augment batch images with flipped image
    Input:  images:          (n, w, d, 3)
            measurement:     (n, )
    """
    # data augmentation: Flipped image
    
    assert images.shape[0] == measurement.shape[0]
    
    fImages = []
    fMeasurement = []
    
    for i, img in enumerate(images):
        image_flipped = np.fliplr(img)
        measurement_flipped = - measurement[i]
#         newImages = np.concatenate((newImages, image_flipped)) 
#         newMeasurement = np.concatenate((newMeasurement, measurement_flipped))
        fImages.append(image_flipped)
        fMeasurement.append(measurement_flipped)
        
    newImages = np.array(fImages)
    newMeasurement = np.array(fMeasurement)
        
    return newImages, newMeasurement


In [ ]:
# # impath = xCenter[0]
# # img = cv.imread(impath)
# images = []
# for i, ipath in enumerate(xCenter):
#     img = cv.imread(ipath)
#     images.append(img)
# #     print(i, img)

# X = np.array(images)
# y = np.array(y)
# # plt.imshow(img)
# print(X.shape)
# print(y.shape)

# timg = cv.imread(xCenter[100])
# ## red and blue colors are inverted
# timg2 = cv.cvtColor(timg, cv.COLOR_RGB2BGR)
    
# timg3 = cv.cvtColor(timg, cv.COLOR_BGR2RGB)

# timg4 =  cv.cvtColor(cv.imread(xCenter[3]), cv.COLOR_BGR2RGB)
# plt.imshow(timg4)



In [ ]:
# nim = len(xCenter)
# t1 = []
# t2 = []
# for i in range(nim):
#     yC = ysteer[i]
#     yL = ysteer[i] + 0.2
#     yR = ysteer[i] - 0.2
#     t1.extend([yC, yL, yR])
#     t2.append([yC, yL, yR])

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=0)
# print(X_train.shape)
# print(X_val.shape)
# print(y_train.shape)

print(np.array(t2))

In [27]:
t = X[1:10,:,:,:]
t.shape

(9, 160, 320, 3)

In [35]:
t = np.concatenate((t,t))
t.shape


(36, 160, 320, 3)

In [4]:
X, y = read3camerasImg(xCenter, xLeft, xRight, steering=ysteer, steerAdj=0.2)



0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [43]:
# randomly select sample and add flipped
X[[0,3],].shape

(2, 160, 320, 3)

In [56]:
from random import sample

n = X.shape[0]

#sample without replacement. add 1/4 sample where each image is flipped
select = sample(range(n), int(n/4))



In [10]:
print(X.shape)
print(y.shape)

# Save example images in example directory
expath = "example/"
if not os.path.isdir(expath): os.makedirs(expath)

# Note: cv has default BGR. As I read with BGR and write with BGR, no need of color conversion
imgC = cv.imread(xCenter[123])
imgL = cv.imread(xLeft[123])
imgR = cv.imread(xRight[123])
cv.imwrite(expath + "cameraLeft.png", imgL)
cv.imwrite(expath + "cameraRight.png", imgR)
cv.imwrite(expath + "cameraCenter.png", imgC)


(27300, 160, 320, 3)
(27300,)


True

In [57]:
x1, y1 = flipBatchImg(X[select,], y[select,])
X = np.concatenate((X,x1))
y = np.concatenate((y,y1))

In [58]:
print(X.shape)
print(y.shape)

(34125, 160, 320, 3)
(34125,)


In [59]:
# Simple Network
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D



model = Sequential()


# Data preprocessing. Lambda normalize. Improve substantially
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape = (160,320,3)) )


# LeNet architecture

# 5x5 convolution, 6 filters
# keras.layers.Conv2D(filters, kernel_size, strides=(1, 1), padding='valid', 
#                     data_format=None, dilation_rate=(1, 1), activation=None, use_bias=True, 
#                     kernel_initializer='glorot_uniform', bias_initializer='zeros', 
#                     kernel_regularizer=None, bias_regularizer=None, activity_regularizer=None, 
#                     kernel_constraint=None, bias_constraint=None)

model.add(Conv2D(9, (10,10), padding='valid', activation = "relu"))
model.add(MaxPooling2D())

# 5x5 convolution, 6 filters
model.add(Conv2D(27, (6,6), padding='valid', activation = None))
model.add(Conv2D(81, (3,3), padding='valid', activation = "relu"))
model.add(MaxPooling2D())

# Flat layer 
model.add(Flatten())

# activation xw +b
model.add(Dense(150))
model.add(Dense(50))
model.add(Dense(1))

model.compile(loss="mse", optimizer="adam")

model.fit(X, y, validation_split=0.2, shuffle=True, epochs = 5)
model.save("model1.h5")


Train on 27300 samples, validate on 6825 samples
Epoch 1/5
   32/27300 [..............................] - ETA: 6:12:44 - loss: 0.0658

KeyboardInterrupt: 

In [ ]:
help(Flatten)

In [ ]:
len(xCenter)

In [ ]:
assert len(xCenter) == len(xLeft) == len(xRight)

In [ ]:
print(xCenter)
print(xLeft[0])
print(xRight[0])
print(y[9])

In [ ]:
for i in range(9):
    print(i)

In [ ]:
images3 = []
images3.extend([imgC, imgL, imgR])


In [ ]:
tmp =np.array(images3)

In [ ]:
tmp.shape

In [ ]:
y[1]